In [4]:
# DAFTAR lIBRARY

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pickle
import optuna
import os
import logging
import time
import warnings
import itertools
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
from matplotlib.dates import DateFormatter

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error

from uuid import uuid4



In [2]:
df = pd.read_csv('student_monnitoring_data.csv')

In [3]:
display((df))
display(df.info())
display(df.describe())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level
0,1,2024-12-01,9:00-15:00,Late,0.92,7.6,6,6,Low
1,1,2024-12-02,8:00-16:00,Late,1.17,6.0,6,2,Medium
2,1,2024-12-03,11:00-14:00,Late,4.56,6.3,4,8,High
3,1,2024-12-04,11:00-16:00,Late,3.07,9.0,2,10,Low
4,1,2024-12-05,9:00-13:00,Absent,3.93,7.4,9,4,High
...,...,...,...,...,...,...,...,...,...
14995,500,2024-12-26,9:00-16:00,Late,1.30,7.2,7,10,Low
14996,500,2024-12-27,9:00-15:00,Absent,1.07,7.9,4,6,High
14997,500,2024-12-28,11:00-14:00,Absent,1.67,7.2,3,5,High
14998,500,2024-12-29,11:00-14:00,Late,0.99,7.2,10,9,Medium


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Student ID          15000 non-null  int64  
 1   Date                15000 non-null  object 
 2   Class Time          15000 non-null  object 
 3   Attendance Status   15000 non-null  object 
 4   Stress Level (GSR)  15000 non-null  float64
 5   Sleep Hours         15000 non-null  float64
 6   Anxiety Level       15000 non-null  int64  
 7   Mood Score          15000 non-null  int64  
 8   Risk Level          15000 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 1.0+ MB


None

,Student ID,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score
count,15000.00000,15000.000000,15000.000000,15000.000000,15000.000000
mean,250.50000,2.762538,6.996780,5.546867,5.471533
std,144.34209,1.301927,1.150973,2.870323,2.868984
min,1.00000,0.500000,5.000000,1.000000,1.000000
25%,125.75000,1.640000,6.000000,3.000000,3.000000
50%,250.50000,2.760000,7.000000,6.000000,5.000000
75%,375.25000,3.900000,8.000000,8.000000,8.000000
max,500.00000,5.000000,9.000000,10.000000,10.000000


In [4]:
# Pengecekan Missing value pada setiap kolom
missing_values = df.isna().sum()
print("Jumlah Missing Value pada setiap kolom:")
print(missing_values)

# Pengecekan Presentasi Missing value per kolom
missing_percentage = (df.isna().sum() / len(df)) * 100
print("\nPersentase Missing value pada setiap Kolim:")
print(missing_percentage)


Jumlah Missing Value pada setiap kolom:
Student ID            0
Date                  0
Class Time            0
Attendance Status     0
Stress Level (GSR)    0
Sleep Hours           0
Anxiety Level         0
Mood Score            0
Risk Level            0
dtype: int64

Persentase Missing value pada setiap Kolim:
Student ID            0.0
Date                  0.0
Class Time            0.0
Attendance Status     0.0
Stress Level (GSR)    0.0
Sleep Hours           0.0
Anxiety Level         0.0
Mood Score            0.0
Risk Level            0.0
dtype: float64


In [5]:
# Menghitung Z-Score untuk kolom-kolom numerik
import numpy as np
z_scores = zscore(df[['Stress Level (GSR)', 'Anxiety Level', 'Mood Score', 'Sleep Hours']])

# Menyaring data dengan Z-Score > 3 atau < -3
outliers_z = np.where(abs(z_scores) > 3)  # Menemukan posisi outlier berdasarkan Z-Score

# Menampilkan data outlier berdasarkan Z-Score
outliers_z_data = df.iloc[outliers_z[0]]
print("Outliers detected by Z-Score:")
print(outliers_z_data)


Outliers detected by Z-Score:
Empty DataFrame
Columns: [Student ID, Date, Class Time, Attendance Status, Stress Level (GSR), Sleep Hours, Anxiety Level, Mood Score, Risk Level]
Index: []


In [6]:
# Menghitung Z-Score per user
def calculate_zscore_per_user(df, columns):
    outliers = []  # Untuk menyimpan hasil outlier per user
    for user_id in df['Student ID'].unique():  # Loop melalui setiap user
        user_data = df[df['Student ID'] == user_id][columns]
        z_scores = zscore(user_data, axis=0)  # Menghitung Z-Score per kolom untuk user tertentu

        # Mencari outlier untuk Z-Score > 3 atau < -3
        outliers_for_user = user_data[(abs(z_scores) > 3).any(axis=1)]  # Filter baris dengan Z-Score > 3
        outliers.append(outliers_for_user)

    # Gabungkan semua outliers per user
    return pd.concat(outliers)

# Tentukan kolom yang ingin dianalisis (kolom numerik)
columns_to_check = ['Stress Level (GSR)', 'Anxiety Level', 'Mood Score', 'Sleep Hours']

# Temukan outliers untuk seluruh dataset
outliers_z_data_per_user = calculate_zscore_per_user(df, columns_to_check)

print("Outliers detected by Z-Score per user:")
print(outliers_z_data_per_user)


Outliers detected by Z-Score per user:
Empty DataFrame
Columns: [Stress Level (GSR), Anxiety Level, Mood Score, Sleep Hours]
Index: []


In [7]:
# Mengonversi kolom 'Date' menjadi format datetime
df['Date'] = pd.to_datetime(df['Date'])
print(df['Date'].head())
df['Day'] = df['Date'].dt.day

df = df.sort_values(by=['Student ID', 'Date'])

user_day_count = df.groupby('Student ID')['Date'].count()

0   2024-12-01
1   2024-12-02
2   2024-12-03
3   2024-12-04
4   2024-12-05
Name: Date, dtype: datetime64[ns]


In [8]:
# label Encoding untuk Risk Level
le = LabelEncoder()
df['Risk Level'] = le.fit_transform(df['Risk Level'])


# label Encoder Untuk ateendande Status
attendance_mapping = {'Present': 0, 'Absent': 1, 'Late': 2}
df['Attendance Status (Data)'] = df['Attendance Status'].map(attendance_mapping)
df['Attendance Status (Data)'] = df['Attendance Status (Data)'].astype('Int64')
df[['Attendance Status', 'Attendance Status (Data)']].head()
display(df.head())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data)
0,1,2024-12-01,9:00-15:00,Late,0.92,7.6,6,6,1,1,2
1,1,2024-12-02,8:00-16:00,Late,1.17,6.0,6,2,2,2,2
2,1,2024-12-03,11:00-14:00,Late,4.56,6.3,4,8,0,3,2
3,1,2024-12-04,11:00-16:00,Late,3.07,9.0,2,10,1,4,2
4,1,2024-12-05,9:00-13:00,Absent,3.93,7.4,9,4,0,5,1


In [9]:

# Memilih kolom untuk Min-Max Scaling dan Standardization
cols_min_max = ['Stress Level (GSR)', 'Anxiety Level', 'Mood Score']
cols_standardize = ['Sleep Hours']

# Membuat objek scaler untuk Min-Max dan Standardization
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Melakukan Min-Max Scaling pada kolom yang dipilih
df[cols_min_max] = min_max_scaler.fit_transform(df[cols_min_max])

# Melakukan Standardization pada kolom yang dipilih
df[cols_standardize] = standard_scaler.fit_transform(df[cols_standardize])

# Menampilkan hasil setelah scaling dan standardisasi
display(df.head())


,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data)
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1


In [10]:
# Lag Features untuk Stress Level (GSR) dan Sleep Hours
df['lag_1_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].shift(1)
df['lag_1_sleep'] = df.groupby('Student ID')['Sleep Hours'].shift(1)

# Rolling Statistics untuk Stress Level (GSR) dengan window size 3 (misalnya 3 hari)
df['rolling_mean_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].rolling(window=3).mean().reset_index(0, drop=True)
df['rolling_std_stress'] = df.groupby('Student ID')['Stress Level (GSR)'].rolling(window=3).std().reset_index(0, drop=True)

# Menampilkan hasil setelah menambahkan lag features dan rolling statistics
display(df.head())

,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data),lag_1_stress,lag_1_sleep,rolling_mean_stress,rolling_std_stress
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2,NaN,NaN,NaN,NaN
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2,0.093333,0.524113,NaN,NaN
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2,0.148889,-0.866061,0.381481,0.451829
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2,0.902222,-0.605403,0.540741,0.377584
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1,0.571111,1.740515,0.745185,0.166212


In [11]:
from statsmodels.tsa.stattools import adfuller

# Fungsi untuk uji ADF pada setiap Student ID
def check_stationarity(data, col):
    result = adfuller(df[col].dropna())
    return result[1]  # Mengembalikan p-value

# Menguji stasioneritas untuk setiap Student ID
p_values = df.groupby('Student ID').apply(lambda group: check_stationarity(group, 'Stress Level (GSR)'))


In [12]:
from statsmodels.tsa.stattools import adfuller

# Fungsi untuk mengecek stasioneritas
def check_stationarity(data, col):
    result = adfuller(data[col].dropna())
    return result[1]  # Mengembalikan p-value

# Fungsi untuk melakukan differencing
def difference_data(data, col, d=1):
    data_copy = data.copy()
    for _ in range(d):
        data_copy[col] = data_copy[col].diff()
    return data_copy

# Inisialisasi variabel
stationary_ids = []
non_stationary_ids = []
p_values_initial = {}
p_values_after_diff = {}
df_processed = []

# Loop per Student ID
for student_id in df['Student ID'].unique():
    data_sub = df[df['Student ID'] == student_id].copy()

    # Cek stasioneritas awal
    p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
    p_values_initial[student_id] = p_value

    if p_value <= 0.05:
        # Sudah stasioner
        stationary_ids.append(student_id)
        p_values_after_diff[student_id] = p_value
    else:
        # Belum stasioner, lakukan differencing pertama
        data_sub = difference_data(data_sub, 'Stress Level (GSR)', d=1)
        p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
        p_values_after_diff[student_id] = p_value

        if p_value <= 0.05:
            stationary_ids.append(student_id)
        else:
            # Lakukan differencing kedua
            data_sub = difference_data(data_sub, 'Stress Level (GSR)', d=2)
            p_value = check_stationarity(data_sub, 'Stress Level (GSR)')
            p_values_after_diff[student_id] = p_value

            if p_value <= 0.05:
                stationary_ids.append(student_id)
            else:
                non_stationary_ids.append(student_id)

    df_processed.append(data_sub)

# Tampilkan daftar hasil akhir
print("\n===== HASIL UJI STASIONERITAS =====")
print("Student ID yang STASIONER:")
print(stationary_ids)

print("\nStudent ID yang BELUM STASIONER:")
print(non_stationary_ids)



===== HASIL UJI STASIONERITAS =====
Student ID yang STASIONER:
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 2

In [13]:
# Menggunakan interpolasi linear untuk mengisi NaN setelah differencing
df[['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score',
      'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']] = \
    df[['Stress Level (GSR)', 'Sleep Hours', 'Anxiety Level', 'Mood Score',
          'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']].interpolate(method='linear')

# Menampilkan data setelah interpolasi
df.head()


,Student ID,Date,Class Time,Attendance Status,Stress Level (GSR),Sleep Hours,Anxiety Level,Mood Score,Risk Level,Day,Attendance Status (Data),lag_1_stress,lag_1_sleep,rolling_mean_stress,rolling_std_stress
0,1,2024-12-01,9:00-15:00,Late,0.093333,0.524113,0.555556,0.555556,1,1,2,NaN,NaN,NaN,NaN
1,1,2024-12-02,8:00-16:00,Late,0.148889,-0.866061,0.555556,0.111111,2,2,2,0.093333,0.524113,NaN,NaN
2,1,2024-12-03,11:00-14:00,Late,0.902222,-0.605403,0.333333,0.777778,0,3,2,0.148889,-0.866061,0.381481,0.451829
3,1,2024-12-04,11:00-16:00,Late,0.571111,1.740515,0.111111,1.000000,1,4,2,0.902222,-0.605403,0.540741,0.377584
4,1,2024-12-05,9:00-13:00,Absent,0.762222,0.350341,0.888889,0.333333,0,5,1,0.571111,1.740515,0.745185,0.166212


In [14]:
# Menghitung jumlah observasi asli untuk setiap Student ID
jumlah_observasi = df.groupby('Student ID').size()

print("\n===== JUMLAH OBSERVASI PER STUDENT ID =====")
for student_id, jumlah in jumlah_observasi.items():
    print(f"Student ID {student_id}: {jumlah} observasi")



===== JUMLAH OBSERVASI PER STUDENT ID =====
Student ID 1: 30 observasi
Student ID 2: 30 observasi
Student ID 3: 30 observasi
Student ID 4: 30 observasi
Student ID 5: 30 observasi
Student ID 6: 30 observasi
Student ID 7: 30 observasi
Student ID 8: 30 observasi
Student ID 9: 30 observasi
Student ID 10: 30 observasi
Student ID 11: 30 observasi
Student ID 12: 30 observasi
Student ID 13: 30 observasi
Student ID 14: 30 observasi
Student ID 15: 30 observasi
Student ID 16: 30 observasi
Student ID 17: 30 observasi
Student ID 18: 30 observasi
Student ID 19: 30 observasi
Student ID 20: 30 observasi
Student ID 21: 30 observasi
Student ID 22: 30 observasi
Student ID 23: 30 observasi
Student ID 24: 30 observasi
Student ID 25: 30 observasi
Student ID 26: 30 observasi
Student ID 27: 30 observasi
Student ID 28: 30 observasi
Student ID 29: 30 observasi
Student ID 30: 30 observasi
Student ID 31: 30 observasi
Student ID 32: 30 observasi
Student ID 33: 30 observasi
Student ID 34: 30 observasi
Student ID 3

In [15]:
#Menyimpan dataset baru setelah preprocessing 
df.to_csv('dataset_baru.csv', index=False)


In [2]:
warnings.filterwarnings("ignore")

# =============================================================================
# BAGIAN 1: FUNGSI DAN KELAS MODEL
# =============================================================================

def setup_logging(model_dir, experiment_name):
    """Mengatur logging untuk menyimpan output ke file."""
    os.makedirs(model_dir, exist_ok=True)
    log_file = os.path.join(model_dir, f'log_{experiment_name}.log')
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

def create_sequences(data, student_ids, target_cols, timesteps=3):
    """Membuat sekuens data untuk model time series."""
    X, X_ids, y = [], [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i + timesteps])
        X_ids.append(student_ids[i + timesteps])
        y.append(data[i + timesteps, -len(target_cols):])
    return np.array(X), np.array(X_ids), np.array(y)

class MultiHeadAttention(nn.Module):
    """Wrapper untuk nn.MultiheadAttention dari PyTorch."""
    def __init__(self, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads"
        self.mha = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, batch_first=True)

    def forward(self, lstm_outputs):
        attn_output, attn_weights = self.mha(lstm_outputs, lstm_outputs, lstm_outputs)
        context_vector = attn_output[:, -1, :]
        return context_vector, attn_weights.mean(dim=1)

class LSTMAttentionModel(nn.Module):
    """Model LSTM yang diintegrasikan dengan Multi-Head Attention Mechanism."""
    def __init__(self, input_dim, num_students, embed_dim, hidden_dim, num_heads, timesteps=3, dropout_rate=0.4):
        super(LSTMAttentionModel, self).__init__()
        self.embedding = nn.Embedding(num_students, embed_dim)
        self.lstm = nn.LSTM(input_dim + embed_dim, hidden_dim, batch_first=True)
        self.attention = MultiHeadAttention(hidden_dim, num_heads)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.risk_head = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(), nn.Dropout(dropout_rate - 0.1 if dropout_rate > 0.1 else 0.0), nn.Linear(hidden_dim // 2, 3))
        self.fc_residual = nn.Linear(hidden_dim, 1)

    def forward(self, x, student_ids):
        if student_ids.ndim == 1: student_ids = student_ids.unsqueeze(1)
        embed = self.embedding(student_ids.squeeze(1))
        embed = embed.unsqueeze(1).repeat(1, x.shape[1], 1)
        x = torch.cat([x, embed], dim=2)
        lstm_out, _ = self.lstm(x)
        context_vector, _ = self.attention(lstm_out)
        out = self.bn(context_vector)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        residual_pred = self.fc_residual(out)
        return risk_logits, residual_pred.squeeze(1)

class LSTMModel(nn.Module):
    """Model LSTM standar tanpa Attention Mechanism."""
    def __init__(self, input_dim, num_students, embed_dim, hidden_dim, timesteps=3, dropout_rate=0.4):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(num_students, embed_dim)
        self.lstm = nn.LSTM(input_dim + embed_dim, hidden_dim, batch_first=True)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.risk_head = nn.Sequential(nn.Linear(hidden_dim, hidden_dim // 2), nn.ReLU(), nn.Dropout(dropout_rate - 0.1 if dropout_rate > 0.1 else 0.0), nn.Linear(hidden_dim // 2, 3))
        self.fc_residual = nn.Linear(hidden_dim, 1)

    def forward(self, x, student_ids):
        if student_ids.ndim == 1: student_ids = student_ids.unsqueeze(1)
        embed = self.embedding(student_ids.squeeze(1))
        embed = embed.unsqueeze(1).repeat(1, x.shape[1], 1)
        x = torch.cat([x, embed], dim=2)
        lstm_out, (h_n, _) = self.lstm(x)
        last_hidden_state = h_n.squeeze(0)
        out = self.bn(last_hidden_state)
        out = self.dropout(out)
        risk_logits = self.risk_head(out)
        residual_pred = self.fc_residual(out)
        return risk_logits, residual_pred.squeeze(1)

# =============================================================================
# BAGIAN 2: FUNGSI TUNING DAN PERAMALAN
# =============================================================================

# ### FUNGSI DIREVISI ###
def tune_arima(train_data, p_range=range(0, 4), d_range=range(0, 3), q_range=range(0, 4)):
    """Mencari parameter ARIMA terbaik menggunakan time series cross-validation."""
    best_mae = float('inf')
    # Inisialisasi dengan None untuk menunjukkan belum ada parameter terbaik yang ditemukan
    best_params = None
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    # Membuat daftar semua kombinasi parameter untuk diiterasi
    pdq = list(itertools.product(p_range, d_range, q_range))
    
    for order in pdq:
        try:
            mae_list = []
            # Validasi silang untuk setiap kombinasi parameter
            for train_index, val_index in tscv.split(train_data):
                train_fold = train_data.iloc[train_index]
                val_fold = train_data.iloc[val_index]
                
                # Hanya gunakan kolom target untuk ARIMA
                model = ARIMA(train_fold['Stress Level (GSR)'], order=order).fit()
                predictions = model.forecast(steps=len(val_fold))
                mae = mean_absolute_error(val_fold['Stress Level (GSR)'], predictions)
                mae_list.append(mae)
            
            mean_mae = np.mean(mae_list)
            
            # Jika MAE rata-rata lebih baik, perbarui parameter terbaik
            if mean_mae < best_mae:
                best_mae = mean_mae
                best_params = order
        except Exception:
            # Lanjutkan ke kombinasi berikutnya jika terjadi error
            continue
            
    # Jika tidak ada parameter yang ditemukan, kembalikan default yang aman
    if best_params is None:
        logging.warning("Grid search ARIMA tidak menemukan parameter yang valid. Menggunakan default (1,1,1).")
        return (1, 1, 1), best_mae
        
    return best_params, best_mae


def find_best_arima_for_feature(series):
    """Mencari parameter ARIMA terbaik untuk satu fitur menggunakan AIC."""
    best_aic = float('inf')
    best_order = None
    
    p_range = d_range = q_range = range(0, 3)  
    pdq = list(itertools.product(p_range, d_range, q_range))
    
    for order in pdq:
        try:
            model = ARIMA(series, order=order, enforce_stationarity=False, enforce_invertibility=False).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = order
        except Exception:
            continue
            
    return best_order if best_order is not None else (1, 1, 1)

def create_feature_forecasters(data, primary_feature_cols):
    """Membuat model peramalan untuk setiap fitur primer dengan mencari order ARIMA terbaik."""
    forecasters = {}
    for col in primary_feature_cols:
        logging.info(f"Mencari order ARIMA terbaik untuk fitur: {col}...")
        try:
            best_order = find_best_arima_for_feature(data[col])
            logging.info(f"Order ARIMA terbaik untuk {col} adalah {best_order}.")
            model = ARIMA(data[col], order=best_order).fit()
            forecasters[col] = model
            logging.info(f"Feature forecaster dibuat untuk {col}")
        except Exception as e:
            logging.error(f"Tidak dapat membuat forecaster untuk {col}: {e}")
            forecasters[col] = None
    return forecasters
    
def tune_lstm(X_train_tensor, X_train_ids_tensor, y_train_risk, y_train_residual, timesteps, num_students, device, use_attention):
    """Mencari hyperparameter terbaik untuk model LSTM menggunakan Optuna."""
    def objective(trial):
        hidden_dim = trial.suggest_int('hidden_dim', 64, 256, step=8)
        dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
        lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
        risk_weight = trial.suggest_float('risk_weight', 0.8, 2.0)
        embed_dim = trial.suggest_int('embed_dim', 10, 50)
        
        if use_attention:
            num_heads = trial.suggest_categorical('num_heads', [2, 4, 8])
            model = LSTMAttentionModel(input_dim=X_train_tensor.shape[2], num_students=num_students, embed_dim=embed_dim, hidden_dim=hidden_dim, num_heads=num_heads, timesteps=timesteps, dropout_rate=dropout_rate).to(device)
        else:
            model = LSTMModel(input_dim=X_train_tensor.shape[2], num_students=num_students, embed_dim=embed_dim, hidden_dim=hidden_dim, timesteps=timesteps, dropout_rate=dropout_rate).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        train_dataset = TensorDataset(X_train_tensor, X_train_ids_tensor, y_train_risk, y_train_residual)
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        
        model.train()
        for epoch in range(100):
            for batch_x, batch_ids, batch_y_risk, batch_y_residual in train_loader:
                batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
                optimizer.zero_grad()
                risk_logits, residual_pred = model(batch_x, batch_ids)
                loss = risk_weight * nn.CrossEntropyLoss()(risk_logits, batch_y_risk) + nn.MSELoss()(residual_pred, batch_y_residual)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            risk_logits, residual_pred = model(X_train_tensor, X_train_ids_tensor)
            final_loss = (risk_weight * nn.CrossEntropyLoss()(risk_logits, y_train_risk) + nn.MSELoss()(residual_pred, y_train_residual)).item()
        return final_loss
        
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=15, n_jobs=-1)
    return study.best_params

def forecast_future(data, arima_stress_fit, model_lstm, le_student, feature_forecasters, timesteps, n_forecast, feature_cols, primary_feature_cols):
    """Meramalkan tingkat stres secara iteratif."""
    logging.info("Memulai peramalan bertingkat yang dinamis...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_lstm.to(device).eval()

    all_forecasts = []

    for student_id in data['Student ID'].unique():
        student_history = data[data['Student ID'] == student_id].copy().reset_index(drop=True)
        if len(student_history) < timesteps:
            continue
            
        student_id_encoded = le_student.transform([student_id])[0]
        student_future_predictions = []
        arima_forecasts = arima_stress_fit.get_forecast(steps=n_forecast).predicted_mean

        future_features_forecasts = {}
        for col, forecaster in feature_forecasters.items():
            if forecaster:
                future_features_forecasts[col] = forecaster.get_forecast(steps=n_forecast).predicted_mean
            else:
                future_features_forecasts[col] = pd.Series([student_history[col].iloc[-1]] * n_forecast, index=arima_forecasts.index)

        with torch.no_grad():
            for i in range(n_forecast):
                input_data = student_history.tail(timesteps)
                current_features_list = [col for col in feature_cols if col in input_data.columns]
                
                if len(current_features_list) != len(feature_cols):
                    # This check might need refinement based on how you handle missing features in student_history
                    # For now, it will break the loop if any feature is missing from the last 'timesteps' rows
                    break

                input_features_sequence = input_data[current_features_list].values.astype(np.float32)
                input_residuals_sequence = input_data['residual'].values.astype(np.float32).reshape(-1, 1)
                input_aug = np.concatenate((input_features_sequence, input_residuals_sequence), axis=1)
                
                current_sequence = torch.tensor(input_aug, dtype=torch.float32).unsqueeze(0).to(device)
                current_id_tensor = torch.tensor([student_id_encoded], dtype=torch.long).to(device)

                _, lstm_residual_pred = model_lstm(current_sequence, current_id_tensor)
                lstm_residual_pred = lstm_residual_pred.cpu().item()
                final_stress_pred = arima_forecasts.iloc[i] + lstm_residual_pred
                student_future_predictions.append(final_stress_pred)

                last_date = student_history['Date'].iloc[-1]
                new_row_data = {
                    'Date': last_date + pd.DateOffset(days=1),
                    'Student ID': student_id,
                    'Student ID Encoded': student_id_encoded,
                    'Stress Level (GSR)': final_stress_pred,
                    'residual': lstm_residual_pred,
                }
                for col in primary_feature_cols:
                    new_row_data[col] = future_features_forecasts[col].iloc[i]

                temp_df = pd.DataFrame([new_row_data])
                student_history = pd.concat([student_history, temp_df], ignore_index=True)
                student_history['lag_1_stress'] = student_history['Stress Level (GSR)'].shift(1)
                if 'Sleep Hours' in student_history.columns:
                    student_history['lag_1_sleep'] = student_history['Sleep Hours'].shift(1)
                student_history['rolling_mean_stress'] = student_history['Stress Level (GSR)'].rolling(window=7, min_periods=1).mean()
                student_history['rolling_std_stress'] = student_history['Stress Level (GSR)'].rolling(window=7, min_periods=1).std()
                
                student_history.ffill(inplace=True)
                student_history.bfill(inplace=True)

        if student_future_predictions:
            forecast_dates = pd.to_datetime([data[data['Student ID']==student_id]['Date'].max() + pd.DateOffset(days=j) for j in range(1, len(student_future_predictions) + 1)])
            forecast_df_student = pd.DataFrame({'Date': forecast_dates, 'Student ID': student_id, 'Forecasted Stress Level': student_future_predictions})
            all_forecasts.append(forecast_df_student)

    if not all_forecasts: return None
    return pd.concat(all_forecasts, ignore_index=True)


# =============================================================================
# BAGIAN 3: FUNGSI UTAMA EKSPERIMEN
# =============================================================================

def run_experiment(data, experiment_name, model_dir, feature_cols, primary_feature_cols, use_attention, timesteps=3):
    """Menjalankan satu siklus eksperimen penuh."""
    setup_logging(model_dir, experiment_name)
    logging.info(f"===== MEMULAI EKSPERIMEN: {experiment_name} =====")
    
    data_exp = data.copy()
    data_exp['Date'] = pd.to_datetime(data_exp['Date'])
    data_exp = data_exp.sort_values(by=['Student ID', 'Date']).reset_index(drop=True)
    
    le_student = LabelEncoder()
    data_exp['Student ID Encoded'] = le_student.fit_transform(data_exp['Student ID'])
    with open(os.path.join(model_dir, f'label_encoder_{experiment_name}.pkl'), 'wb') as f: pickle.dump(le_student, f)
    
    unique_dates = sorted(data_exp['Date'].unique())
    split_point = int(len(unique_dates) * 0.8)
    train_dates, test_dates = unique_dates[:split_point], unique_dates[split_point:]
    
    train_data = data_exp[data_exp['Date'].isin(train_dates)].copy().reset_index(drop=True)
    test_data = data_exp[data_exp['Date'].isin(test_dates)].copy().reset_index(drop=True)
    
    logging.info(f"Data split: {len(train_data)} baris training, {len(test_data)} baris testing.")
    if len(test_data) == 0 or len(train_data) == 0: raise ValueError("Data latih atau uji kosong.")

    target_cols = ['Stress Level (GSR)']
    for col in feature_cols + target_cols:
        for df in [train_data, test_data]:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)
            df[col] = df.groupby('Student ID')[col].transform(lambda x: x.ffill().bfill())
    train_data.fillna(train_data.mean(numeric_only=True), inplace=True)
    test_data.fillna(train_data.mean(numeric_only=True), inplace=True)

    logging.info("Tuning dan melatih model ARIMA...")
    best_params_arima, _ = tune_arima(train_data)
    logging.info(f"Parameter ARIMA terbaik: {best_params_arima}")
    
    arima_stress_fit = ARIMA(train_data['Stress Level (GSR)'], order=best_params_arima).fit()
    with open(os.path.join(model_dir, f'arima_stress_model_{experiment_name}.pkl'), 'wb') as f: pickle.dump(arima_stress_fit, f)

    train_data['residual'] = train_data['Stress Level (GSR)'] - arima_stress_fit.predict(start=train_data.index[0], end=train_data.index[-1])
    test_forecast = arima_stress_fit.get_forecast(steps=len(test_data)).predicted_mean
    test_data['residual'] = test_data['Stress Level (GSR)'].values - test_forecast.values
    
    train_values = train_data[feature_cols + target_cols].values
    test_values = test_data[feature_cols + target_cols].values
    train_student_ids = train_data['Student ID Encoded'].values
    test_student_ids = test_data['Student ID Encoded'].values
    X_train, X_train_ids, y_train = create_sequences(train_values, train_student_ids, target_cols, timesteps)
    X_test, X_test_ids, y_test = create_sequences(test_values, test_student_ids, target_cols, timesteps)
    
    residual_train_seq = train_data.loc[timesteps:, 'residual'].values[:len(X_train)].astype(np.float32)
    residual_test_seq = test_data.loc[timesteps:, 'residual'].values[:len(X_test)].astype(np.float32)
    
    X_train_aug = np.concatenate((X_train[:, :, :len(feature_cols)], residual_train_seq[:, np.newaxis, np.newaxis].repeat(timesteps, axis=1)), axis=2).astype(np.float32)
    X_test_aug = np.concatenate((X_test[:, :, :len(feature_cols)], residual_test_seq[:, np.newaxis, np.newaxis].repeat(timesteps, axis=1)), axis=2).astype(np.float32)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"Tuning LSTM (use_attention={use_attention})...")
    
    y_train_risk_tensor = torch.tensor(y_train[:, 0], dtype=torch.long).to(device)
    y_train_residual_tensor = torch.tensor(residual_train_seq).to(device)
    best_params_lstm = tune_lstm(torch.tensor(X_train_aug).to(device), torch.tensor(X_train_ids).to(device), y_train_risk_tensor, y_train_residual_tensor, timesteps, len(le_student.classes_), device, use_attention)
    logging.info(f"Parameter LSTM Terbaik: {best_params_lstm}")

    if use_attention:
        model_lstm = LSTMAttentionModel(input_dim=X_train_aug.shape[2], num_students=len(le_student.classes_), embed_dim=best_params_lstm['embed_dim'], hidden_dim=best_params_lstm['hidden_dim'], num_heads=best_params_lstm['num_heads'], timesteps=timesteps, dropout_rate=best_params_lstm['dropout_rate']).to(device)
    else:
        model_lstm = LSTMModel(input_dim=X_train_aug.shape[2], num_students=len(le_student.classes_), embed_dim=best_params_lstm['embed_dim'], hidden_dim=best_params_lstm['hidden_dim'], timesteps=timesteps, dropout_rate=best_params_lstm['dropout_rate']).to(device)

    optimizer = torch.optim.Adam(model_lstm.parameters(), lr=best_params_lstm['lr'], weight_decay=1e-5)
    train_dataset = TensorDataset(torch.tensor(X_train_aug), torch.tensor(X_train_ids), torch.tensor(y_train[:, 0], dtype=torch.long), torch.tensor(residual_train_seq))
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    
    logging.info("Memulai pelatihan akhir model LSTM...")
    # List to store loss values per epoch for plotting
    epoch_losses = [] 
    for epoch in range(50):
        model_lstm.train()
        total_loss = 0
        for batch_x, batch_ids, batch_y_risk, batch_y_residual in train_loader:
            batch_x, batch_ids, batch_y_risk, batch_y_residual = batch_x.to(device), batch_ids.to(device), batch_y_risk.to(device), batch_y_residual.to(device)
            optimizer.zero_grad()
            risk_logits, residual_pred = model_lstm(batch_x, batch_ids)
            loss = best_params_lstm['risk_weight'] * nn.CrossEntropyLoss()(risk_logits, batch_y_risk) + nn.MSELoss()(residual_pred, batch_y_residual)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        epoch_losses.append(avg_loss)
        logging.info(f"Epoch {epoch+1}/{50}, Loss: {avg_loss:.4f}")
            
    torch.save(model_lstm.state_dict(), os.path.join(model_dir, f'lstm_model_{experiment_name}.pth'))

    # Save the epoch losses to a file for later plotting
    with open(os.path.join(model_dir, f'lstm_training_losses_{experiment_name}.pkl'), 'wb') as f:
        pickle.dump(epoch_losses, f)

    model_lstm.eval()
    with torch.no_grad():
        lstm_residual_pred = model_lstm(torch.tensor(X_test_aug).to(device), torch.tensor(X_test_ids).to(device))[1].cpu().numpy()
    
    arima_test_pred_aligned = test_forecast.values[timesteps:][:len(lstm_residual_pred)]
    hybrid_test_pred = arima_test_pred_aligned + lstm_residual_pred
    y_true_test = y_test[:, -1]
    
    hybrid_mae = mean_absolute_error(y_true_test, hybrid_test_pred)
    hybrid_rmse = np.sqrt(mean_squared_error(y_true_test, hybrid_test_pred))
    logging.info(f"Kinerja Hybrid {experiment_name}: MAE={hybrid_mae:.4f}, RMSE={hybrid_rmse:.4f}")

    logging.info("Membuat peramal fitur untuk peramalan masa depan...")
    full_historical_data = pd.concat([train_data, test_data]).sort_values('Date')
    feature_forecasters = create_feature_forecasters(full_historical_data, primary_feature_cols)
    
    future_forecasts = forecast_future(full_historical_data, arima_stress_fit, model_lstm, le_student, feature_forecasters, timesteps, 10, feature_cols, primary_feature_cols)

    logging.info("Membuat grafik...")
    predictions_df = pd.DataFrame({
        'Date': test_data['Date'].iloc[timesteps:].iloc[:len(y_true_test)], 
        'Student ID': test_data['Student ID'].iloc[timesteps:].iloc[:len(y_true_test)], 
        'Predicted Stress Level': hybrid_test_pred, 
        'Actual Stress Level': y_true_test
    })

    for student_id in data_exp['Student ID'].unique():
        plt.figure(figsize=(15, 7))
        student_actual_data = data_exp[data_exp['Student ID'] == student_id]
        plt.plot(student_actual_data['Date'], student_actual_data['Stress Level (GSR)'], label='Actual Stress Level', color='blue', linestyle='-')
        
        student_pred_data = predictions_df[predictions_df['Student ID'] == student_id]
        if not student_pred_data.empty:
            plt.plot(student_pred_data['Date'], student_pred_data['Predicted Stress Level'], label='Hybrid Prediction', color='green', linestyle='--')
            residuals = student_pred_data['Actual Stress Level'] - student_pred_data['Predicted Stress Level']
            plt.plot(student_pred_data['Date'], residuals, label='Residuals', color='red', linestyle=':')
        
        if future_forecasts is not None:
            student_forecast_data = future_forecasts[future_forecasts['Student ID'] == student_id]
            if not student_forecast_data.empty:
                plt.plot(student_forecast_data['Date'], student_forecast_data['Forecasted Stress Level'], color='orange', marker='o', linestyle='--', label='Dynamic Forecast')
        
        plt.title(f'Stress Level Prediction for Student {student_id}\nExperiment: {experiment_name}')
        plt.xlabel('Date')
        plt.ylabel('Stress Level (GSR)')
        plt.legend()
        plt.grid(True)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
        plt.gcf().autofmt_xdate(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(model_dir, f'plot_{experiment_name}_student_{student_id}.png'))
        plt.close()

    logging.info(f"===== EKSPERIMEN SELESAI: {experiment_name} =====")
    return {'mae_stress': hybrid_mae, 'rmse_stress': hybrid_rmse, 'epoch_losses': epoch_losses} # Return epoch_losses

def plot_final_metrics(results, output_dir):
    """Membuat bar chart yang membandingkan metrik dari semua eksperimen."""
    experiment_names = list(results.keys())
    mae_scores = [res['mae_stress'] for res in results.values()]
    rmse_scores = [res['rmse_stress'] for res in results.values()]

    x = np.arange(len(experiment_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(x - width/2, mae_scores, width, label='MAE')
    rects2 = ax.bar(x + width/2, rmse_scores, width, label='RMSE')

    ax.set_ylabel('Scores')
    ax.set_title('Perbandingan Metrik Kinerja antar Eksperimen')
    ax.set_xticks(x)
    ax.set_xticklabels(experiment_names, rotation=25, ha="right")
    ax.legend()

    ax.bar_label(rects1, padding=3, fmt='%.4f')
    ax.bar_label(rects2, padding=3, fmt='%.4f')

    fig.tight_layout()
    plt.savefig(os.path.join(output_dir, 'comparison_metrics.png'))
    plt.close()
    print(f"Grafik perbandingan metrik disimpan di '{output_dir}/comparison_metrics.png'")

def plot_loss_curves(all_results, output_dir):
    """Membuat grafik loss untuk setiap eksperimen."""
    logging.info("Membuat grafik kurva loss untuk setiap eksperimen...")
    for exp_name, metrics in all_results.items():
        if 'epoch_losses' in metrics and metrics['epoch_losses']:
            plt.figure(figsize=(10, 6))
            plt.plot(metrics['epoch_losses'], label=f'{exp_name} Training Loss', color='purple')
            plt.title(f'Training Loss Over Epochs for {exp_name}')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'loss_curve_{exp_name}.png'))
            plt.close()
            logging.info(f"Grafik kurva loss untuk {exp_name} disimpan di '{output_dir}/loss_curve_{exp_name}.png'")
        else:
            logging.warning(f"Tidak ada data loss ditemukan untuk eksperimen: {exp_name}")

# =============================================================================
# BAGIAN 4: EKSEKUSI EKSPERIMEN
# =============================================================================

if __name__ == "__main__":
    try:
        data_main = pd.read_csv('dataset_baru.csv')
        data_main['Date'] = pd.to_datetime(data_main['Date'])
        data_main = data_main.sort_values(by=['Student ID', 'Date']).reset_index(drop=True)
        data_main['lag_1_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].shift(1)
        data_main['lag_1_sleep'] = data_main.groupby('Student ID')['Sleep Hours'].shift(1)
        data_main['rolling_mean_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].rolling(window=7, min_periods=1).mean().reset_index(0,drop=True)
        data_main['rolling_std_stress'] = data_main.groupby('Student ID')['Stress Level (GSR)'].rolling(window=7, min_periods=1).std().reset_index(0,drop=True)
        data_main = data_main.groupby('Student ID').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
    except FileNotFoundError:
        print("Error: File 'dataset_baru.csv' tidak ditemukan.")
        exit()

    features_full = ['Sleep Hours', 'Anxiety Level', 'Mood Score', 'lag_1_stress', 'lag_1_sleep', 'rolling_mean_stress', 'rolling_std_stress']
    primary_features_full = ['Sleep Hours', 'Anxiety Level', 'Mood Score']
    features_reduced = ['Anxiety Level', 'Mood Score', 'rolling_mean_stress']
    primary_features_reduced = ['Anxiety Level', 'Mood Score']

    base_output_dir = "hasil_eksperimen_final"
    os.makedirs(base_output_dir, exist_ok=True)

    experiments = [
        {"name": "1_Hybrid_NoAttn_FullFeatures", "dir": os.path.join(base_output_dir, "1_no_attn_full"), "features": features_full, "primary_features": primary_features_full, "use_attention": False},
        {"name": "2_Hybrid_WithAttn_FullFeatures", "dir": os.path.join(base_output_dir, "2_with_attn_full"), "features": features_full, "primary_features": primary_features_full, "use_attention": True},
        {"name": "3_Hybrid_WithAttn_ReducedFeatures", "dir": os.path.join(base_output_dir, "3_with_attn_reduced"), "features": features_reduced, "primary_features": primary_features_reduced, "use_attention": True}
    ]

    total_start_time = time.time()
    all_results = {}
    
    for exp in experiments:
        print(f"\n{'='*80}\nMEMULAI EKSPERIMEN: {exp['name']}\n{'='*80}")
        exp_start_time = time.time()
        results = run_experiment(data=data_main, experiment_name=exp['name'], model_dir=exp['dir'], feature_cols=exp['features'], primary_feature_cols=exp['primary_features'], use_attention=exp['use_attention'])
        all_results[exp['name']] = results
        exp_end_time = time.time()
        duration = exp_end_time - exp_start_time
        print(f"--- HASIL UNTUK: {exp['name']} ---")
        if results:
            print(f"  - MAE : {results['mae_stress']:.4f}")
            print(f"  - RMSE: {results['rmse_stress']:.4f}")
        print(f"Waktu Eksekusi: {duration/60:.2f} menit")
        
    total_duration = time.time() - total_start_time
    print(f"\n{'#'*80}\nSEMUA EKSPERIMEN SELESAI\nTotal Waktu Eksekusi: {total_duration/60:.2f} menit\n{'#'*80}")
    
    if all_results:
        plot_final_metrics(all_results, base_output_dir)
        plot_loss_curves(all_results, base_output_dir)


MEMULAI EKSPERIMEN: 1_Hybrid_NoAttn_FullFeatures


[I 2025-07-09 15:23:29,980] A new study created in memory with name: no-name-7c1cf55c-1266-4f24-8082-8e8526c63b49
[I 2025-07-09 15:40:24,044] Trial 9 finished with value: 0.8210300207138062 and parameters: {'hidden_dim': 64, 'dropout_rate': 0.4987036546765926, 'lr': 0.007590430479735875, 'risk_weight': 1.945849032819175, 'embed_dim': 20}. Best is trial 9 with value: 0.8210300207138062.
[I 2025-07-09 15:41:28,808] Trial 10 finished with value: 0.6541939377784729 and parameters: {'hidden_dim': 192, 'dropout_rate': 0.23428870257605872, 'lr': 0.00018632033296342408, 'risk_weight': 1.348539465491759, 'embed_dim': 26}. Best is trial 10 with value: 0.6541939377784729.
[I 2025-07-09 15:41:34,687] Trial 11 finished with value: 0.20377343893051147 and parameters: {'hidden_dim': 120, 'dropout_rate': 0.3149256470607706, 'lr': 0.0011410889437556008, 'risk_weight': 1.9646494152772715, 'embed_dim': 17}. Best is trial 11 with value: 0.20377343893051147.
[I 2025-07-09 15:41:39,340] Trial 0 finished wit

--- HASIL UNTUK: 1_Hybrid_NoAttn_FullFeatures ---
  - MAE : 0.0458
  - RMSE: 0.0573
Waktu Eksekusi: 27.77 menit

MEMULAI EKSPERIMEN: 2_Hybrid_WithAttn_FullFeatures


[I 2025-07-09 15:51:14,051] A new study created in memory with name: no-name-6829198a-ff38-4b03-aacb-6ceb1c7fa0ed
[I 2025-07-09 16:16:48,986] Trial 0 finished with value: 0.0565929189324379 and parameters: {'hidden_dim': 80, 'dropout_rate': 0.4281054335485103, 'lr': 0.0010845982754715004, 'risk_weight': 1.26508136521313, 'embed_dim': 36, 'num_heads': 4}. Best is trial 0 with value: 0.0565929189324379.
[I 2025-07-09 16:17:00,915] Trial 3 finished with value: 0.386491984128952 and parameters: {'hidden_dim': 64, 'dropout_rate': 0.3858979122634386, 'lr': 0.004666302471622624, 'risk_weight': 1.1165191965988224, 'embed_dim': 26, 'num_heads': 2}. Best is trial 0 with value: 0.0565929189324379.
[I 2025-07-09 16:17:03,966] Trial 11 finished with value: 0.16396038234233856 and parameters: {'hidden_dim': 64, 'dropout_rate': 0.3451849687563834, 'lr': 0.003942041307270116, 'risk_weight': 1.600962342603191, 'embed_dim': 26, 'num_heads': 4}. Best is trial 0 with value: 0.0565929189324379.
[I 2025-07-

--- HASIL UNTUK: 2_Hybrid_WithAttn_FullFeatures ---
  - MAE : 0.0383
  - RMSE: 0.0470
Waktu Eksekusi: 38.60 menit

MEMULAI EKSPERIMEN: 3_Hybrid_WithAttn_ReducedFeatures


[I 2025-07-09 16:30:16,103] A new study created in memory with name: no-name-86cb0838-4a87-4bb7-97e8-06f6818c2423
[I 2025-07-09 16:51:13,748] Trial 1 finished with value: 0.2510125935077667 and parameters: {'hidden_dim': 72, 'dropout_rate': 0.35778803028747935, 'lr': 0.002401608231850295, 'risk_weight': 1.3486756131642794, 'embed_dim': 17, 'num_heads': 8}. Best is trial 1 with value: 0.2510125935077667.
[I 2025-07-09 16:51:22,789] Trial 13 finished with value: 0.13997648656368256 and parameters: {'hidden_dim': 80, 'dropout_rate': 0.3164274351884703, 'lr': 0.0021490508546645995, 'risk_weight': 1.8317256643008581, 'embed_dim': 48, 'num_heads': 8}. Best is trial 13 with value: 0.13997648656368256.
[I 2025-07-09 16:51:39,584] Trial 5 finished with value: 0.3696882128715515 and parameters: {'hidden_dim': 80, 'dropout_rate': 0.42774375142811216, 'lr': 0.002930164264201446, 'risk_weight': 1.0341923358424308, 'embed_dim': 42, 'num_heads': 8}. Best is trial 13 with value: 0.13997648656368256.
[

--- HASIL UNTUK: 3_Hybrid_WithAttn_ReducedFeatures ---
  - MAE : 0.0404
  - RMSE: 0.0487
Waktu Eksekusi: 32.30 menit

################################################################################
SEMUA EKSPERIMEN SELESAI
Total Waktu Eksekusi: 98.67 menit
################################################################################
Grafik perbandingan metrik disimpan di 'hasil_eksperimen_final_loss/comparison_metrics.png'
